# P-MOPSS: Pipeline for Magellan Optical Planetary Spectra Survey #

UNDER CONSTRUCTION. Will allow all reduction to be run from this notebook!


In [1]:
import numpy as np
#import os

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

from astropy.io import fits

from astropy.time import Time

from datetime import datetime

from setup import *
from FullFrame import FullFrame

##--------------------------------------------------------------------------##
##             NECESSARY USER INPUTS BELOW....                              ##
##--------------------------------------------------------------------------##
obs_date='ut20150819'                           #observation date
obj_name='Wasp4'                                #object name                      
data_path='/Volumes/ermay_ext/Magellan/'+obs_date+'/'+obj_name+'_Spec/'      #path to where your data is saved
flat_path='/Volumes/ermay_ext/Magellan/'+obs_date+'/'+obj_name+'_Flats/'     #path to where your SLIT flats are saved
DARKS='/Volumes/ermay_ext/Magellan/'+obs_date+'/'+obj_name+'_Darks/'     #path to where your darks are saved
FLATS='/Volumes/ermay_ext/Magellan/'+obs_date+'/Full_Flats/'             #path to where the full field flats are saved 



# --------------------------------------------------------------------------- 
# The following cell extracts header information.

You can add arrays in HeaderData.py following the methods there. Certain values only need to be read out for 1 chip


In [ ]:
## ------------------------------------------------------------- ##
## Run this block to read header and save relevant information.  ##
## REQUIRED for observation times and number                     ##
## ------------------------------------------------------------- ##
from HeaderData import *
ReadHeader(data_path)

## This cell only needs to be run ONCE. Will save to a .npz file you can read in for the arrays
## Keys in .npz file are the names of the arrays:
##'------------------------------------------'
##'   Observing Times: obs_times[n_exp]'
##'   Object Airmass:  airmass[n_exp]'
##'   Read Noise:      elc_noise[chip]'
##'   CCD Temperature: ccd_temp[chip,n_exp]'
##'   Structure Temp:  stc_temp[chip_n_exp]'
##'   Ion Pump Press:  ion_pump[chip_n_exp]'
##'------------------------------------------'

## NOTES:
##  This cell can take a few minutes to run. I'm working on speeding up the header reading process

# --------------------------------------------------------------------------- 
# Master Dark and Flat Frames

Master Darks and Flats are first created by stitching together two chips in the vertical direction.

They are then combined into 'Full Frame' 8-chip Master Dark and Flats

In [ ]:
## ------------------------------------------------------------- ##
##     make sure your paths are updated. (first cell)            ##
## ------------------------------------------------------------- ##

print 'Location of Darks is currently set to: ', DARKS
print 'Location of Flats is currently set to: ', FLATS
print ' '

from MasterFrame import MasterFrame

dark=MasterFrame(DARKS,'Darks')                #first keyword=path, second keyword=save name
dark_full=FullFrame(1,dark)                    #number of frames, matrix of double-chip images
del dark                                       #clearing memory


flat=MasterFrame(FLATS,'Flats')
flat_full=FullFrame(1,flat)
flat_full/=np.nanmedian(flat_full)
del flat

fig,ax=plt.subplots(1,2,figsize=(16.,8.))
ax[0].imshow(dark_full[0,:,:],cmap=plt.cm.Greys_r)      #plotting to make sure combined right
ax[0].set_title('Full Frame Darks')
ax[1].imshow(flat_full[0,:,:],cmap=plt.cm.Greys_r)
ax[1].set_title('Full Frame Flats')

# --------------------------------------------------------------------------- 
# Creates Object Masks

(these are used to later only extract the parts of the data we want)

First: Objects are located using the flat_thres variable

Second: Vertical Chips are Stitched

Third: Horizontal Chips are Stitched

Fourth: Objects are Combined across Vertical Chips. 

In [ ]:
## ------------------------------------------------------------- ##
## Masks are simply matrices of 1s and 0s                        ##
## ------------------------------------------------------------- ##

## user input needed:
flat_root='ift0016c'         #root name of a slit flat image
flat_thres=6000.             #flat threshold: number of counts for 'edges' of masks.

from FindMasks import FindMasks
from FindMasks import CombineMasks
#from FullFrame import FullFrame

masks=FindMasks(flat_path,flat_root,flat_thres)
mask_full=FullFrame(1,masks)
del masks

print' -->> Combining Masks'
mask_edges=CombineMasks(mask_full)
print'          (done)'


# NOTES:
# Some improvements to be made, still. Works fine for now
# I need to find a way to ignore 0th order boxes. These can be trimmed later, for now
# this will be updated to be dependent on the median level of counts found

# --------------------------------------------------------------------------- 
# Define Objects: Label Target and Remove 0th-order


In [ ]:
## ------------------------------------------------------------- ##
##  This rearranges masks so that the [0] element is the target  ##
##  zero-th order masks are also removed.                        ##
## ------------------------------------------------------------- ##

## using numbers output on combined mask image from cell above,define the following:
TAR=7                           #index for target
FIR=np.array([11,12,13,14])     #indices for zeroth order masks. These will be deleted


################## don't change below  ################
mask_edges=np.load('SaveData/CombinedMasks.npz')['mask_edges']
masks_fin=np.empty([mask_edges.shape[0]-len(FIR),mask_edges.shape[1]])*0.0

masks_fin[0,:]=mask_edges[TAR,:]
j=1
for i in range(0,mask_edges.shape[0]):
    if not i in FIR and i!=7:
        masks_fin[j,:]=mask_edges[i,:]
        j+=1
        
np.savez('SaveData/FinalMasks.npz',masks=masks_fin)
del masks_fin

# --------------------------------------------------------------------------- 
# Extract 2D Spectra for each Object #
Dark is subtracted, Flat is divided out.

In [ ]:
from Extract2d import Extract2D

data=Extract2D(data_path)

#output is a dictionary. Keys are formated as 'obj#', with 'obj0' being the target and 'obj1'+ being calibrators
#
#invalid value encountered in divide error - IGNORE. We are dividing full frame images which include NaNs for the gaps.

 -->> Loading Masks
          (done)
 -->> Loading Darks
              572.0
          (done)
 -->> Loading Flats
              1.0
          (done)
 -->> Loading HeaderData
               112
          (done)


Extract2d.py:80: RuntimeWarning: invalid value encountered in divide
  image_full/=flat_full[0,:,:]


           ( SAVED DATA FOR IMAGE  1 )
           ( SAVED DATA FOR IMAGE  2 )
           ( SAVED DATA FOR IMAGE  3 )
           ( SAVED DATA FOR IMAGE  4 )
           ( SAVED DATA FOR IMAGE  5 )
           ( SAVED DATA FOR IMAGE  6 )
           ( SAVED DATA FOR IMAGE  7 )
           ( SAVED DATA FOR IMAGE  8 )
           ( SAVED DATA FOR IMAGE  9 )
           ( SAVED DATA FOR IMAGE  10 )
           ( SAVED DATA FOR IMAGE  11 )
           ( SAVED DATA FOR IMAGE  12 )
           ( SAVED DATA FOR IMAGE  13 )
           ( SAVED DATA FOR IMAGE  14 )
           ( SAVED DATA FOR IMAGE  15 )
           ( SAVED DATA FOR IMAGE  16 )
           ( SAVED DATA FOR IMAGE  17 )
           ( SAVED DATA FOR IMAGE  18 )
           ( SAVED DATA FOR IMAGE  19 )
           ( SAVED DATA FOR IMAGE  20 )
           ( SAVED DATA FOR IMAGE  21 )
           ( SAVED DATA FOR IMAGE  22 )
           ( SAVED DATA FOR IMAGE  23 )
           ( SAVED DATA FOR IMAGE  24 )
           ( SAVED DATA FOR IMAGE  25 )
         

# --------------------------------------------------------------------------- 
# Flatten 2D Spectra #

Fits a Background function and subtracts off.

Fits a gaussian to each line of the 2D Spectra to dectect the center.

The aperture is currently done as follows: Median aperture across lambda for each frame. Varies in time. Calcualted as 3*(median(FWHM)) of the guassian fit 

# --------------------------------------------------------------------------- 
# Wavelength Calibration #

Not working in notebook, will eventually update this code to run this way. 

Currently: Run via command line. I'll include instructions for this part soon.

# --------------------------------------------------------------------------- 
# Time Correltation and Apply Wavelength Calibration #
This accounts for shifts in the spectral direction by oversampling the spectra by your given factor and cross correlating in time to dectect shifts down to (1/factor) of a pixel.

After correlation, all spectra are shifted as necessary and wavelength calibration is applied.